In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
from scipy.spatial import distance_matrix
import scipy.spatial as spatial
import statistics 
import time
from sklearn import metrics
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances
#from sklearn.neighbors.nearest_centroid import NearestCentroid

from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import Birch
from sklearn.cluster import OPTICS, cluster_optics_dbscan

In [39]:
raw_data=pd.read_csv('cpu4_0.10.csv')

m=raw_data.iloc[:,0:2]


,timestamp,value
0,1469376000,0.847300
1,1469376300,-0.036137
2,1469376600,0.074292
3,1469376900,0.074292
4,1469377200,-0.036137


In [3]:
def sortedtop (m, K):

    #distance matrix from every point to every point
    dis=pd.DataFrame(squareform(pdist(m, 'euclidean')))

    sortedtop = []

    for col in dis.columns:
        topK=dis[col].nsmallest(K) #Closest K samples to point in each column 
        top=max(topK) #KNN to point
        sortedtop.append(top)

    #Distances of KNN in asceding order
    KNNdist=pd.DataFrame(sortedtop)
    KNNdist=KNNdist.rename(columns={0:'KNNdist'})

    results=pd.concat([KNNdist, m], axis=1)
    results['index'] = results.index #previously not here
    results_sort=results.sort_values(by=['KNNdist'])
    results_sort=results_sort.reset_index(drop=True)
    last=len(m.columns)+2 #previously +1
    m_sort=results_sort.iloc[:,1:last]
    
    return results_sort, m_sort

In [4]:
def params (results_sort, m_sort, densLVL):

    #sumdist=results_sort['KNNdist'].sum()
    #radius=sumdist/len(results_sort)
    
    KNNdist=pd.DataFrame(results_sort.iloc[:,0])
    #n=len(KNNdist)
    n=math.ceil(len(KNNdist)-0.05*len(KNNdist))
    woTOP = results_sort.iloc[0:n,:]
    sumdist=woTOP['KNNdist'].sum()
    #radius=sumdist/len(woTOP)
    radius=woTOP['KNNdist'].std()
    

    check=[]
    alldist=[]
    ii=[]
    yy=[]

    
    #KNNdist=pd.DataFrame(results_sort.iloc[:,0])
    KNNdist2 = KNNdist.values #previously deleted

    
    #n=math.ceil(len(KNNdist)-0.1*len(KNNdist)) #exclude top 10% of distances

    for i in range (0,n-1):
        if i not in check:
            y=KNNdist.iat[i,0]
            point_tree = spatial.cKDTree(KNNdist2)
            droppts=point_tree.query_ball_point(KNNdist2[i], radius)
            for m in range (0,len(droppts)):
                check.append(droppts[m])

            w=i
            while w in check and w < len(results_sort):
                w += 1

            x=KNNdist.iloc[w,0]
            changedist=x-y
            alldist.append(changedist)
            ii.append(i)
            yy.append(y)

    new=results_sort.iloc[ii]
    index1=pd.DataFrame(new['index'])
    index=index1.reset_index(drop=True)
    pts1=pd.DataFrame({"KNNdist":yy, "i":ii})
    pts=pd.concat([pts1,index], axis=1)
    alldist=pd.DataFrame(data=alldist)
    alldist.columns=['KNNchange']
    distpts=alldist.join(pts)

    nlarge=distpts.nlargest(densLVL,['KNNchange'])
    nlargestsort=nlarge.sort_values(by=['i'])
    epsALL=pd.DataFrame(nlargestsort.iloc[:,1]).reset_index(drop=True)
    epsidx=pd.DataFrame(nlargestsort.iloc[:,2]).reset_index(drop=True)
    idx=pd.DataFrame(nlargestsort.iloc[:,3]).reset_index(drop=True)

    return epsALL, epsidx, idx


In [5]:
UNCLASSIFIED = False
NOISE = None

def _dist(p,q):
    return math.sqrt(np.power(p-q,2).sum())

def _eps_neighborhood(p,q,eps):
    return _dist(p,q) < eps

def _region_query(m_sort, point_id, eps):
    n_points = m_sort.shape[0]
    seeds = []
    for i in range(0, n_points):
        if _eps_neighborhood(m_sort.iloc[point_id,:], m_sort.iloc[i,:], eps): #if point i is in neighborhood of point_id
            seeds.append(i) #
    return seeds

In [6]:
def loop2cluster (m_sort, classifications, point_id, cluster_id, eps, epsALL, min_points, epsidx):            
    seeds = _region_query(m_sort, point_id, eps) #all neighbors of 'point_id'
    #n_points = m.shape[1]
    if len(seeds) < min_points: # number of point in neighborhood less than min points
        classifications[point_id] = UNCLASSIFIED #stay unclassified
        return classifications,cluster_id,eps,min_points,m_sort
    else:
        classifications[point_id] = cluster_id #label point_id we're comparing too
        for seed_id in seeds: 
            classifications[seed_id] = cluster_id #label all neighbors to point_id
        index=seeds
        
        while len(seeds) > 0:
            current_point = seeds[0]
            results = _region_query(m_sort, current_point, eps) # checking for reachable points
            if len(results) >= min_points:
                for i in range(0, len(results)):
                    result_point = results[i]
                    if classifications[result_point] == UNCLASSIFIED:
                        seeds.append(result_point)
                    classifications[result_point] = cluster_id # if reachable point then add to the same cluster
                    #m=np.delete(m,[point_id, seeds[m], results[i]],1)
            seeds = seeds[1:]
            
        cluster_id = cluster_id + 1

    return classifications,cluster_id,eps,min_points,m_sort

In [ ]:
def purity_score(clusters, classes):

    A = np.c_[(clusters,classes)]
    n_accurate = 0.
    
    for j in np.unique(A[:,0]):
        z = A[A[:,0] == j, 1]
        x = np.argmax(np.bincount(z))
        n_accurate += len(z[z == x])
        
    return n_accurate / A.shape[0]

In [34]:
def knndbscan(m, K):
    min_points = K
    results_sort, m_sort =  sortedtop (m, K)
    #densLVL = round(math.sqrt(K))
    densLVL = 3
    epsALL, epsidx, idx = params (results_sort, m_sort, densLVL)
    row = 0
    eps = epsALL.iat[row,0]
    cluster_id = 1 #initial cluster
    n_points = m_sort.shape[0] #total number of points in dataset
    classifications = [UNCLASSIFIED] * n_points #initialize all points to unclassified
    check = []

    #while n_points-np.count_nonzero(classifications) > math.ceil(0.15*n_points): # keep looping until 5% of data is left

    for point_id in range(0, n_points):
        if row < len(epsALL) and len(epsALL) > 1:
            if point_id <= epsidx.iat[row,0]:
                eps = epsALL.iat[row,0]
                    #min_points = K-5
            else:
                row += 1
            if classifications[point_id] == UNCLASSIFIED and point_id not in check: #and k>=cluster_id: #if point is unclassified
                classifications,cluster_id,eps,min_points,m_sort \
                = loop2cluster(m_sort, classifications, point_id, cluster_id, eps, epsALL, min_points, epsidx)
                
        check.append(point_id)
            
    return m_sort, classifications, results_sort, idx

In [ ]:
start = time.time()
KNN=3

m_sort, classifications, results_sort, idx = knndbscan(m, KNN)
end = time.time()

cl = pd.DataFrame(classifications, columns=['classes'])
index = m_sort['index']
classification = pd.concat([cl, index], axis=1)

print(end - start)